# AutoGen Solution

## Load Azure Configurations

In [1]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client
Using the model client class

In [2]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)

## Creating the Agents

In [3]:
from typing import Sequence
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.agents.web_surfer import MultimodalWebSurfer
from datetime import date


# Get the current date
current_date = date.today()

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=az_model_client,
    system_message=f"""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        Multimodal Web Surfer: Searches for information in the internet on the weather advisory for the current date: {current_date} on the hiking location.
        Hiking Agent: Provides a hiking plan for the hiker based on the weather advisory.
        Product Agent: Recommends products from its file search tool.
        Writing Agent: Writes a document combining the hiking plan, product suggestions, and weather advisory.
        Critique Agent: Critiques the hiking plan, product agent, and writing agent and suggests improvements.
        
        
    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>
    """,
)

web_surfer_agent = MultimodalWebSurfer(
    name="MultimodalWebSurfer",
    model_client=az_model_client,
    description="A web surfer agent that searches for information on the internet.",
)

hiking_agent = AssistantAgent(
    "HikingAgent",
    description="A Hiking agent. Writes a hiking plan based on the weather advisory.",
    model_client=az_model_client,
    system_message="""
    You are a Hiking agent.
    You will create a hiking plan based on the weather advisory.
    After writing the hiking plan, you will pass it to the Critique agent for review.
    """,
)

critique_agent = AssistantAgent(
    "CritiqueAgent",
    description="A critique agent. Critiques the Hiking plan, product suggestions, and writing agent.",
    model_client=az_model_client,
    system_message="""
    You are a Critique agent.
    You need to ensure that the hiking plan is well thought out and provides constructive feedback.
    You need to ensure the multi-modal web surfer has provided the correct information on the weather
    You need to ensure that the product agent provides products from file search tool such as the CozyNights Sleeping Bag, BaseCamp Folding Table, and many more.
    You need to ensure that the writing agent has combined the hiking plan, the product suggestions, and weather advisory into a single document.
    You need to ensure feedback from user is incorporated and ask agents to make necessary changes.
    """,
)

# Define a tool to write the content to a file
async def write_to_file(content: str) -> str:
    # Write the content to a file
    with open("output/output.txt", "w") as file:
        file.write(content)

writing_agent = AssistantAgent(
    "WritingAgent",
    description="A Writing agent that combines the hiking plan, product suggestions, and weather advisory into a single document.",
    model_client=az_model_client,
    tools=[write_to_file],
    system_message="""
    You are a writing agent.
    You will combine the hiking plan, the product suggestions, and weather advisory into a single document.
    You will write the final output to a file.
    """,
)

# Create the user proxy agent.
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

In [4]:
from autogen_core import CancellationToken
from autogen_ext.agents.openai import OpenAIAssistantAgent
from autogen_agentchat.messages import TextMessage
from openai import AsyncAzureOpenAI

cancellation_token = CancellationToken()

# Create AzureOpenAI client
client = AsyncAzureOpenAI(azure_endpoint=azure_openai_endpoint, 
                          api_version=azure_openai_api_version, 
                          api_key=azure_openai_key)

#### Create a vector store for file search ####
vector_store = await client.beta.vector_stores.create(name="Hiking Products")
# Specify the folder containing the files
folder_path = "../Data/products/"
# Get all file paths in the folder
file_paths = [os.path.join(folder_path, file_name) for file_name in os.listdir(folder_path)]
# Open file streams
file_streams = [open(path, "rb") for path in file_paths]
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = await client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)
# Close file streams
for file in file_streams:
    file.close()

#### Create an assistant with file search ####
product_agent = OpenAIAssistantAgent(
    name="ProductAgent",
    description="Suggestions for products",
    client=client,
    model=azure_openai_deployment,
    instructions="You will provide possible products for the hiker based on the file search tool such as the TrailMaster X4 Tent, CozyNights Sleeping Bag, and more.",
    tools=["file_search"],
    tool_resources={"file_search":{"vector_store_ids":[vector_store.id]}}
)

In [5]:
# Test the product agent
user_input = TextMessage(source="user", content="What is the price of the TrailMaster X4 Tent?")
response = await product_agent.on_messages([user_input], cancellation_token
)
print(response.chat_message.content)

The price of the TrailMaster X4 Tent is $250【4:0†source】.


## Creating the team
Let’s create the team with two termination conditions: 
- TextMentionTermination to end the conversation when the User Proxy sends “APPROVE”
- MaxMessageTermination to limit the conversation to avoid infinite loop.

In [6]:
text_mention_termination = TextMentionTermination("APPROVE")
max_messages_termination = MaxMessageTermination(max_messages=50)
termination = text_mention_termination | max_messages_termination

team = SelectorGroupChat(
    [planning_agent, web_surfer_agent, hiking_agent, product_agent, writing_agent, critique_agent, user_proxy],
    model_client=az_model_client,
    termination_condition=termination,
)

## Specify the Task and Run the Team

In [7]:
task = "I want to hike in the Grand Canyon, Arizona, USA."
# Use asyncio.run(...) if you are running this in a script.
response = await Console(team.run_stream(task=task))

/home/vscode/.local/lib/python3.10/site-packages/autogen_agentchat/teams/_group_chat/_selector_group_chat.py:135: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(messages=select_speaker_messages)


---------- user ----------
I want to hike in the Grand Canyon, Arizona, USA.


/home/vscode/.local/lib/python3.10/site-packages/autogen_agentchat/agents/_assistant_agent.py:330: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  result = await self._model_client.create(


---------- PlanningAgent ----------
1. Multimodal Web Surfer: Search for information on the weather advisory for the Grand Canyon, Arizona, USA, for the current date: 2025-01-19.
2. Hiking Agent: Based on the weather advisory retrieved by the Multimodal Web Surfer, provide a hiking plan for the Grand Canyon, Arizona, USA.
3. Product Agent: Recommend products suitable for hiking in the Grand Canyon, Arizona, USA, considering the current weather conditions.
4. Writing Agent: Write a document combining the hiking plan, product suggestions, and weather advisory.
5. Critique Agent: Critique the hiking plan, product suggestions, and the document, then suggest improvements.


/home/vscode/.local/lib/python3.10/site-packages/autogen_ext/agents/web_surfer/_multimodal_web_surfer.py:509: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(


---------- MultimodalWebSurfer ----------
I typed 'current weather advisory for Grand Canyon, Arizona, USA' into '0 characters out of 2000'.

 Here is a screenshot of the webpage: [current weather advisory for Grand Canyon, Arizona, USA - Search](https://www.bing.com/search?q=current+weather+advisory+for+Grand+Canyon%2C+Arizona%2C+USA&form=QBLH&sp=-1&lq=0&pq=&sc=0-0&qs=n&sk=&cvid=4A29F6AC43194600A48413E7982723D9&ghsh=0&ghacc=0&ghpl=).
 The viewport shows 26% of the webpage, and is positioned at the top of the page 
The following metadata was extracted from the webpage:

{
    "meta_tags": {
        "referrer": "origin-when-cross-origin",
        "SystemEntropyOriginTrialToken": "A1L3tx5CzccqjN3lK6st/fXMwhf9EeokCPf8XCt0DVI8JPbg37BWq0zKvlqgkdm8YEUbthoGkC/xdR1+iIz4txAAAABxeyJvcmlnaW4iOiJodHRwczovL3d3dy5iaW5nLmNvbTo0NDMiLCJmZWF0dXJlIjoiTXNVc2VyQWdlbnRMYXVuY2hOYXZUeXBlIiwiZXhwaXJ5IjoxNzM5NzI0MzExLCJpc1N1YmRvbWFpbiI6dHJ1ZX0=",
        "og:description": "Intelligent search from Bing makes it 

Selector selected the previous speaker: WritingAgent


---------- WritingAgent ----------
The document has been successfully written to include the pros and cons for each recommended product. Please review it and let me know if any additional adjustments or further information are needed.
---------- CritiqueAgent ----------
### Hiking Plan for Grand Canyon, Arizona, USA (2025-01-19)

**Weather Advisory:**
- **Date:** January 19, 2025
- **Conditions:** Sunny with a cloudless sky
- **Temperature Range:** Maximum of 57.2°F (14°C) and minimum of 19.4°F (-7°C)
- **Recommendations:** Dress in layers to accommodate temperature variations, and ensure preparedness for colder temperatures, especially in the early morning or late evening.

**Trail Information and Safety:**
- **Emergency Contacts:** Local emergency services contacts should be obtained before the hike.
- **Safety Tips:** Stay on marked trails, carry a whistle, and notify someone of your hiking plans.

### Proposed Hiking Itinerary

**Morning:**

1. **Early Start at South Kaibab Trailhe

## Printing whole response per agent source

In [8]:
# ANSI escape code for bold text
bold_start = "\033[1m"
bold_end = "\033[0m"

# ANSI escape code for red text
red_start = "\033[31m"
red_end = "\033[0m"

for messages in response.messages:
    source = messages.source
    print(f"{bold_start}{red_start}{source}{bold_end}{red_end}")
    print(messages.content)

user
I want to hike in the Grand Canyon, Arizona, USA.
PlanningAgent
1. Multimodal Web Surfer: Search for information on the weather advisory for the Grand Canyon, Arizona, USA, for the current date: 2025-01-19.
2. Hiking Agent: Based on the weather advisory retrieved by the Multimodal Web Surfer, provide a hiking plan for the Grand Canyon, Arizona, USA.
3. Product Agent: Recommend products suitable for hiking in the Grand Canyon, Arizona, USA, considering the current weather conditions.
4. Writing Agent: Write a document combining the hiking plan, product suggestions, and weather advisory.
5. Critique Agent: Critique the hiking plan, product suggestions, and the document, then suggest improvements.
MultimodalWebSurfer
['I typed \'current weather advisory for Grand Canyon, Arizona, USA\' into \'0 characters out of 2000\'.\n\n Here is a screenshot of the webpage: [current weather advisory for Grand Canyon, Arizona, USA - Search](https://www.bing.com/search?q=current+weather+advisory+for

## Clean up the resources
Clean up the OpenAI Assistant Agent

In [9]:
# Clean up resources
await product_agent.delete_uploaded_files(cancellation_token)
await product_agent.delete_assistant(cancellation_token)